Importing the libraries and dependencies

In [48]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [49]:
#downloading he stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
#printing the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Loding the dataset from kaggle

In [51]:
#installing the kaggle
!pip install kaggle

In [52]:
#configuring the path of json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [53]:
#api command to fetch the dataset
!kaggle competitions download -c fake-news

fake-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [54]:
#extracting the zip file
from zipfile import ZipFile
dataset = '/content/fake-news.zip'

with ZipFile(dataset,'r')as zip:
  zip.extractall()
  print("dataset is extracted")

dataset is extracted


In [55]:
fn_dataset = pd.read_csv('/content/train.csv')

In [56]:
#printing the row and column of dataset
fn_dataset.shape

(20800, 5)

In [57]:
#printing the first five rows of the dataset
fn_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [58]:
#checking the missing values in each colums
fn_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [59]:
#filling the null values with empty string
fin_dataset =fn_dataset.fillna('')

In [60]:
#merging the author name and title
fn_dataset['target']= fn_dataset['author']+' '+fn_dataset['title']


In [61]:
fn_dataset['target'].dtype

dtype('O')

In [62]:
#seprating the data features and labels
X = fn_dataset.drop(columns = 'label',axis=1)
Y= fn_dataset['label']

In [63]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [64]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


Stemming :
process of reducing the word into root word

In [65]:
#loading the function PorterStemmer
port_stem = PorterStemmer()

In [66]:
def stemming(target):
  stemmed_content = re.sub('[^a-zA-Z]',' ', str(target))
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [67]:
fn_dataset['target'] = fn_dataset['target'].apply(stemming)

In [68]:
print(fn_dataset['target'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: target, Length: 20800, dtype: object


Seprating the columns label and target " as we are feeding these columns to our model"

In [69]:
#seperating the label and target
X = fn_dataset['target'].values
Y = fn_dataset['label'].values

In [70]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [71]:
print(Y)

[1 0 1 ... 0 1 1]


In [72]:
#converting the text data into numerical values , as we know computer cannot understand textual data
vector = TfidfVectorizer()
vector.fit(X)
X =vector.transform(X)

In [73]:
print(X)

  (0, 14627)	0.28553358349485225
  (0, 12568)	0.2563589820798857
  (0, 8310)	0.3596536533885757
  (0, 8048)	0.2934713957861362
  (0, 7190)	0.24644399370485162
  (0, 6552)	0.21885788468666617
  (0, 4637)	0.23132772852542255
  (0, 3543)	0.2689082732577136
  (0, 3359)	0.3596536533885757
  (0, 2757)	0.24749629983427848
  (0, 2312)	0.37305738773975167
  (0, 247)	0.2702588721108487
  (1, 15664)	0.30543189988638736
  (1, 6377)	0.19530026425832656
  (1, 5140)	0.7101106653194483
  (1, 3328)	0.2633913741576576
  (1, 2619)	0.19610930415709218
  (1, 2066)	0.38047138877486064
  (1, 1764)	0.15430291936631466
  (1, 1391)	0.29649664509588236
  (2, 14561)	0.41792081528179686
  (2, 8973)	0.4931379270922205
  (2, 5579)	0.35039256210774505
  (2, 5031)	0.3876367834516749
  (2, 2895)	0.4571578367807032
  :	:
  (20797, 12240)	0.25299971331630106
  (20797, 11516)	0.2746816397264756
  (20797, 11322)	0.2462639038918503
  (20797, 9606)	0.08009780291419748
  (20797, 8942)	0.17302467550398123
  (20797, 8879)	0.292

Splitting the data into training and test data

In [74]:
X_train, X_test, Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Training the model

In [75]:
# loading the model
model = LogisticRegression()


In [76]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation

In [77]:
#accuracy score on training data
X_train_prediction = model.predict(X_train)

accuracy_train = accuracy_score(X_train_prediction,Y_train)

In [78]:
print("accuracy score on training data   :   ",accuracy_train)

accuracy score on training data   :    0.9900841346153846


Evaluation on test data

In [79]:
#accuracy score on training data
X_test_prediction = model.predict(X_test)

accuracy_test = accuracy_score(X_test_prediction,Y_test)

In [80]:
print("accuracy score on training data   :   ",accuracy_test)

accuracy score on training data   :    0.9848557692307692


Make the model to predict the values

In [81]:
news = X_test[5]

predict_value = model.predict(news)

print(predict_value)

if(predict_value[0]==0):
  print("The news is real")
else:
  print("the news is fake")

[1]
the news is fake
